In [2]:
import pdfplumber
import pandas as pd
import io
import json
from openai import OpenAI
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key = api_key)

In [6]:
def extract_text_from_pdf(pdf_path):
    """Extract text from all pages of the PDF."""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text.strip()

In [11]:
def generate_sections(cv_text, tower_selected):
    with open(f"prompt_dictionary/{tower_selected.lower()}.md", "r", encoding="utf-8") as f:
        file = f.read()
    prompt_completed = f""" Generate one structured section (no bullets, bold keywords). 
    {file} 
    
    Candidate CV: {cv_text}
    """

    return prompt_completed
    # response = client.chat.completions.create(
    #     model="gpt-5-mini",
    #     messages=[{"role": "user", "content": prompt_completed, "reasoning-effort": "medium"}],
    # )
    # return response.choices[0].message.content.strip()

In [13]:
tower_selected = 'SAP'
cv_text = extract_text_from_pdf('data/input/candidate_cv.pdf')
output = generate_sections(cv_text, tower_selected)

In [14]:
print(output)

 Generate one structured section (no bullets, bold keywords). 
        You are an AI assistant that summarizes a candidate CV into specific structured sections. Do NOT create new sections.
    Write the output in English.
    Return an dictionary where the section is the Key and the its information is the Value.

    Formatting rules:
    - Use line breaks to separate each idea or item. Do NOT use bullet symbols or dashes.
    - Maintain concise, professional tone.
    - Exclude candidate name, company names, institutions, and dates.
    - Assume Graphik 9 font style.

    Section details:
    - NAME: Candidate’s name.
    - TOWER: Choose ONLY ONE → SAP ARIBA, SAP S/4HANA, CONTROL TOWER or ORACLE.
        • CONTROL TOWER → mentions SQL, Python, R, Power BI, Tableau, Looker, Alteryx, ETL, analytics, ML, automation, software development or data-driven tasks.
        • SAP S/4HANA → SAP modules (FI, CO, MM, SD, PP, PM, etc.), Fiori, ABAP, or S/4HANA implementations.
        • SAP ARIBA → 